In [1]:
"""
This script creates artificial data for a discrete choice problem.
Assume there are three modes of transportation to choose from. Six fixed
variables were designed as significant and five as non-significant.
Seven random variables (five normal, one uniform, one triangular) were
designed as significant.
Three normal variables were correlated.
Two normal variables were non-linearly transformed.
"""

'\nThis script creates artificial data for a discrete choice problem.\nAssume there are three modes of transportation to choose from. Six fixed\nvariables were designed as significant and five as non-significant.\nSeven random variables (five normal, one uniform, one triangular) were\ndesigned as significant.\nThree normal variables were correlated.\nTwo normal variables were non-linearly transformed.\n'

In [2]:
    ``


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import scipy.stats as ss
    

In [6]:
def noise(n_obs, perc=1, random_state=None):
    random_state = random_state or np.random
    noise_vec = random_state.normal(0, 1, n_obs)
    return noise_vec

In [1]:
def random_col(N, P, J, random_state=None):
    rand_nums = random_state.randint(low=5, high=25, size=(3,))/10
    return np.tile(rand_nums, N*P) + 0.5*noise(N*P*J, random_state=random_state)

def generate_random_df(N, P, J, num_fixed=0, num_isvars=0, num_randvars=0, random_state=None):
    df = pd.DataFrame()
    df['choice_id'] = np.repeat(np.arange(1, (N*P+1)), J)
    df['ind_id'] = np.repeat(np.arange(1, N+1), P*J)
    df['alt'] = np.tile(np.arange(1, J+1), N*P)

    varnames = []
    for i in range(num_fixed):
        coef_name = 'added_fixed' + str(i+1)
        varnames.append(coef_name)
        df[coef_name] = random_col(N, P, J, random_state=random_state)

    for i in range(num_isvars):
        coef_name = 'added_isvar' + str(i+1)
        varnames.append(coef_name)
        col_vals = np.repeat(random_state.random(N*P)*100, J)
        for j in range(J):
            if j == 0:
                df[coef_name] = col_vals
            else:
                df[coef_name + "." + str(j+1)] = col_vals

    for i in range(num_randvars):
        coef_name = 'added_random' + str(i+1)
        varnames.append(coef_name)
        df[coef_name] = random_col(N, P, J, random_state=random_state)

    return df, varnames


In [7]:
np.random.seed(0)
N = 600  # Number of observations
P = 20  # Number of choices per individual
J = 1  # Number of alternatives
num_fixed = 11
num_isvars = 0
num_nonsig = 5
num_randvars = 7

random_state = np.random.RandomState(2)

df, varnames = generate_random_df(N, P, J, num_fixed=num_fixed, num_isvars=num_isvars,
                                  num_randvars=num_randvars, random_state=random_state)



df = df.round(8)

ValueError: operands could not be broadcast together with shapes (36000,) (12000,) 

In [ ]:
df.head(10)

In [7]:
# Define coefficients (betas)
# Fixed betas
fixed_coefs = [random_state.choice([-1,1]) * random_state.uniform(.25, 1) for i in range(num_fixed)]
fixed_coefs = np.array(fixed_coefs)

# rewrite old coef names
fixed_coefs[-num_nonsig::] = 0
old_coef_names = varnames[num_fixed-num_nonsig:num_fixed]
varnames[num_fixed-num_nonsig:num_fixed] = ['nonsig' + str(i+1) for i in range(num_nonsig)]

for ii, old_name in enumerate(old_coef_names):
    new_name = 'nonsig' + str(ii+1)
    df = df.rename(columns={old_name: new_name})

fixed_coefs = list(fixed_coefs)
isvar_coefs =  [0 for i in range(num_isvars)]
new_isvars = []
for coef in isvar_coefs:
    for j in range(J):
        isvar_alt = coef + np.random.uniform(0, 1)
        new_isvars.append(isvar_alt)

isvar_coefs = new_isvars
isvar_coefs = np.array(isvar_coefs)
isvar_coefs[np.arange(0, J*num_isvars, J)] = 0
isvar_coefs = list(isvar_coefs)

In [8]:
print(fixed_coefs)
print(isvar_coefs)

[0.433577010307193, -0.416540006686381, 0.7511760870427675, 0.966082816446922, 0.49429543192841774, 0.9239320540275335, 0.0, 0.0, 0.0, 0.0, 0.0]
[]


In [9]:
# Random mean between -1.5 and 1.5, excluding -.1 - .1 as hard to detect effect
random_coefs_mean = [random_state.choice([-1,1]) * random_state.uniform(.5, 1.5) for i in range(num_randvars-2)]
random_coefs_sd = [random_state.uniform(1.0, 1.5) for i in range(num_randvars-2)]

cov_mat = np.diag(random_coefs_sd)
cov_mat[0, 1] = cov_mat[1, 0] = 0.25
cov_mat[0, 2] = cov_mat[2, 0] = 0.4
cov_mat[1, 2] = cov_mat[2, 1] = 0.5

random_coefs_uniform_a = 0
random_coefs_uniform_b = random_state.uniform(1, 2)

random_coefs_tri_left = 0
random_coefs_tri_right = random_state.uniform(1, 2)
random_coefs_tri_mode = random_coefs_tri_right/2

rand_coefs = [np.array([]) for i in range(num_randvars)]

for i in range(N):
    res_normal = random_state.multivariate_normal(random_coefs_mean, cov_mat)
    res_uniform = np.array([random_state.uniform(random_coefs_uniform_a, random_coefs_uniform_b)])
    res_triangular = np.array([random_state.triangular(random_coefs_tri_left, random_coefs_tri_mode, random_coefs_tri_right)])
    res = np.concatenate((res_normal, res_uniform, res_triangular))

    for r in range(num_randvars):
        rand_coefs[r] = np.append(rand_coefs[r], np.repeat(res[r], P*J))


In [10]:
print(random_coefs_mean)
print(random_coefs_sd)

[1.4311152955243416, -0.6952543135912662, 0.7210641910793044, 0.6437314245813626, 1.2029349753939522]
[1.4694441330570271, 1.0618526306860647, 1.2793609519277316, 1.3597765787300091, 1.0082774046874046]


In [11]:
random_coefs_uniform_a, random_coefs_uniform_b

(0, 1.1751241688272402)

In [12]:
random_coefs_tri_left, random_coefs_tri_mode, random_coefs_tri_right

(0, 0.9641799673341549, 1.9283599346683098)

In [13]:
cov_mat

array([[1.46944413, 0.25      , 0.4       , 0.        , 0.        ],
       [0.25      , 1.06185263, 0.5       , 0.        , 0.        ],
       [0.4       , 0.5       , 1.27936095, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.35977658, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.0082774 ]])

In [14]:
B_fixed = [np.repeat(f_coef, P*N*J) for f_coef in fixed_coefs]
B_isvar = [np.tile(isvar_coefs[(i*J):(i*J)+J], P*N) for i in range(num_isvars)]

# Convert betas to matrix for easy product
B = [B_fixed, B_isvar, rand_coefs]
B = [B_i for B_i in B if B_i != []]
B = np.vstack(B).T

In [ ]:
# Visualise values after non-linear transformation
# import matplotlib.pyplot as plt
# plt.hist(inv_boxcox(df['added_random4'], 0.4), bins=30)

In [15]:
# Multiply and generate probability
isvars = ['added_isvar' + str(i+1) for i in range(num_isvars)]

X = df.values[:, 3:]  # Extract only necessary columns
XB = (X*B).sum(axis=1).reshape(N*P, J)
eps = np.random.gumbel(0, 1, (N*P, J))
eXB = np.exp(XB)
prob = eXB/eXB.sum(axis=1, keepdims=True)
# Use monte carlo simulation to predict choice
# y = np.apply_along_axis(lambda p: np.eye(J, dtype='int64')[np.argmax(p)], 1, prob).reshape(N*P*J,)
# y = y.reshape(N*P*J,)

y = []
U = XB + eps

for row in U:
    idx = np.argmax(row)
    row_i = np.zeros(J)
    row_i[idx] = 1
    y.append(row_i)
y = np.array(y)
y = np.squeeze(y.reshape(-1, 1))

df['choice'] = y

# Apply non-linear transformations for boxcox testing
df['added_random4'] = inv_boxcox(df['added_random4'], 0.4)
df['added_random5'] = inv_boxcox(df['added_random5'], 0.3)

# Save to CSV
df.to_csv("artificial_1h_mixed_corr_trans_march2023_MOOF.csv", index=False)

In [16]:
# pre-check frequencies of each alt
np.mean(U, axis=0)

array([11.73011546, 13.34124179, 10.02414039])

In [17]:
model = MixedLogit()

varnames = ['added_fixed1',
 'added_fixed2',
 'added_fixed3',
 'added_fixed4',
 'added_fixed5',
 'added_fixed6',
 #'nonsig1',
 #'nonsig2',
 #'nonsig3',
 #'nonsig4',
 #'nonsig5',
 'added_random1',
 'added_random2',
 'added_random3',
 'added_random4',
 'added_random5',
 'added_random6',
 'added_random7']

X = df[varnames].values
y = df['choice'].values

# # Apply non-linear transformations for boxcox testing
df['added_random4'] = boxcox(df['added_random4'], 0.4)
df['added_random5'] = boxcox(df['added_random5'], 0.3)

transvars = [] # ['added_random4', 'added_random5']
randvars = {'added_random1': 'n', 'added_random2': 'n', 'added_random3': 'n', 'added_random4': 'n',
            'added_random5': 'n', 'added_random6': 'u', 'added_random7': 't'}

correlation = ['added_random1', 'added_random2', 'added_random3']

model.fit(X,
          y,
          ids=df['choice_id'].values,
          panels=df['ind_id'].values,
          varnames=varnames,
          isvars=isvars,
        #   gtol=2e-6,
          ftol=1e-8,
        #   method="L-BFGS-B",
          correlation=correlation,
          transvars=transvars,
          randvars=randvars,
          alts=df['alt']
          )
model.summary()

Optimization terminated successfully.
         Current function value: 7306.235717
         Iterations: 46
         Function evaluations: 57
         Gradient evaluations: 57
Estimation time= 78.0 seconds
Proportion of alternatives: observed choice
[0.27816667 0.60083333 0.121     ]
Proportion of alternatives: predicted choice
[0.27810605 0.60047474 0.1214192 ]
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
added_fixed1         0.4143613975  0.0374781383 11.0560827112             0 ***
added_fixed2        -0.4102532510  0.0353688228 -11.5992905170             0 ***
added_fixed3         0.7498146067  0.0363127812 20.6487793757             0 ***
added_fixed4         0.9113544974  0.0393331664 23.1701279598             0 ***
added_fixed5         0.4564933000  0.0376986237 12.1090176690             0 ***
added_f

In [18]:
model.corr()

correlation matrix
             added_random1  added_random2  added_random3  
added_random1  1.0          0.19611047   0.32148529   
added_random2  0.19611047   1.0          0.39795835   
added_random3  0.32148529   0.39795835   1.0          


In [19]:
model.stddev()

Standard Deviations
added_random1  added_random2  added_random3  added_random4  added_random5  added_random6  added_random7  
 1.13291349    0.9781349     1.103972   0.54900703   0.30161786   0.10307274   0.94651795  


In [ ]:
# np.linalg.norm(model.coeff_[:11] - fixed_coefs)